### 논문
- https://arxiv.org/pdf/2308.15928

### 입력

In [241]:
arr = [-2, -5, 1, -4, 6, -3, 7]

### 전처리

In [242]:
pi = [0] + arr + [len(arr) + 1]
n = pi[-1]
signed_pi_inv = [0] * (2*n)
sign = [False] * (n + 1)
pi_inv = [0] * (n + 1)

signed_pi = [0]
signed_pi_inv[0] = 0
sign[0] = False
pi_inv[0] = 0
pi_inv[n] = n
for i, num in enumerate(arr, 1):
    signed_pi_inv[-num] = len(signed_pi)
    signed_pi.append(-num)
    signed_pi_inv[num] = len(signed_pi)
    signed_pi.append(num)
    sign[abs(num)] = num < 0
    pi_inv[abs(num)] = i
signed_pi_inv[-n] = len(signed_pi)
signed_pi.append(-n)
sign[6] = False

print(pi)
print(sign)
print(pi_inv)
print(signed_pi)
print(signed_pi_inv)

[0, -2, -5, 1, -4, 6, -3, 7, 8]
[False, False, True, True, True, True, False, False, False]
[0, 3, 1, 6, 4, 2, 5, 7, 8]
[0, 2, -2, 5, -5, -1, 1, 4, -4, -6, 6, 3, -3, -7, 7, -8]
[0, 6, 1, 11, 7, 3, 10, 14, 15, 13, 9, 4, 8, 12, 2, 5]


In [243]:
edge_start_at = [None] * (2*n)
edge_end_at = [None] * (2*n)
black = [True] * n

for i in range(0, n):
    s = signed_pi_inv[i]
    e = signed_pi_inv[-(i + 1)]
    if s > e:
        s, e = e, s
    edge_start_at[s] = i
    edge_end_at[e] = i
    black[i] = sign[i] != sign[i + 1]

print(edge_start_at)
print(edge_end_at)
print(black)

[0, 2, 1, 5, 4, None, None, None, 3, None, 6, None, None, None, 7, None]
[None, None, None, None, None, 0, 1, 4, None, 5, None, 3, 2, 6, None, 7]
[False, True, False, False, False, True, False, False]


In [244]:
G = {i: {i} for i in range(n)}
S1 = []
S2 = []
for i in range(0, 2*n):
    if (v := edge_start_at[i]) != None:
        S1.append(v)
    elif (v := edge_end_at[i]) != None:
        while S1[-1] != v:
            S2.append(s := S1.pop())
            G[v].add(s)
            G[s].add(v)
        S1.pop()
        while S2:
            S1.append(S2.pop())

G

{0: {0, 1, 2, 4, 5},
 1: {0, 1, 4, 5},
 2: {0, 2, 6},
 3: {3, 5, 6},
 4: {0, 1, 4},
 5: {0, 1, 3, 5},
 6: {2, 3, 6},
 7: {7}}

### toggle 정의

In [247]:
def toggle(vertices, G, black):
    print('@toggle START')
    print(f'\t{G=}')
    print(f'\t{black=}')
    ret = set(vertices)
    for neighbor in vertices:
        G[neighbor] = G[neighbor] ^ vertices | {neighbor}
        black[neighbor] = not black[neighbor]
    print(f'>>>\n\t{G=}')
    print(f'\t{black=}')
    print('@toggle END')
    return ret

### reversal_sort 정의

In [250]:
def reversal_sort(G, black):
    S1, S2 = [], []
    V = set()
    
    for v in G:
        if len(G[v]) != 1 or black[v]:
            V.add(v)
    
    print(f'initial {V=}\n')
            
    def toggle_black():
        for v in V:
            if black[v]:
                break
        affected = toggle(G[v], G, black)
        for v in set(V):
            if len(G[v]) == 1 and not black[v]:
                V.remove(v)
        S1.append((v, affected))
        print(f'=== toggled, {V=}\n')

    def undo_last_toggle():
        w, to_undo = S1.pop()
        affected = toggle(to_undo, G, black)
        print(f'=== undone toggle, {V=}\n')
        return w, affected

    while any(black[v] for v in V):
        toggle_black()
    
    while V:
        while all(not black[v] for v in V):
            undone = undo_last_toggle()
            S2.append(undone)
        while any(black[v] for v in V):
            toggle_black()
        if not black[S2[-1][0]]:
            undo_last_toggle()
        
    print()
    print(f'{S1=}')
    print(f'{S2=}')
    print(f'{V=}')
    print(f'{G=}')
    print(f'{black=}')

    ret = []
    for sort in S1 + S2[::-1]:
        ret.append(sort[0])
    return ret

### 테스트

In [251]:
sort = reversal_sort(G, black)
print(sort)

initial V={0, 2, 3, 4, 5, 6}

@toggle START
	G={0: {0, 2}, 1: {1}, 2: {0, 2, 6}, 3: {3, 5, 6}, 4: {4, 5}, 5: {3, 4, 5}, 6: {2, 3, 6}, 7: {7}}
	black=[True, False, False, False, True, False, False, False]
>>>
	G={0: {0}, 1: {1}, 2: {2, 6}, 3: {3, 5, 6}, 4: {4, 5}, 5: {3, 4, 5}, 6: {2, 3, 6}, 7: {7}}
	black=[False, False, True, False, True, False, False, False]
@toggle END
=== toggled, V={2, 3, 4, 5, 6}

@toggle START
	G={0: {0}, 1: {1}, 2: {2, 6}, 3: {3, 5, 6}, 4: {4, 5}, 5: {3, 4, 5}, 6: {2, 3, 6}, 7: {7}}
	black=[False, False, True, False, True, False, False, False]
>>>
	G={0: {0}, 1: {1}, 2: {2}, 3: {3, 5, 6}, 4: {4, 5}, 5: {3, 4, 5}, 6: {3, 6}, 7: {7}}
	black=[False, False, False, False, True, False, True, False]
@toggle END
=== toggled, V={3, 4, 5, 6}

@toggle START
	G={0: {0}, 1: {1}, 2: {2}, 3: {3, 5, 6}, 4: {4, 5}, 5: {3, 4, 5}, 6: {3, 6}, 7: {7}}
	black=[False, False, False, False, True, False, True, False]
>>>
	G={0: {0}, 1: {1}, 2: {2}, 3: {3, 5, 6}, 4: {4}, 5: {3, 5}, 6: {3,

In [238]:
sort.extend(reversal_sort(G, black))
print(sort)

initial V=set()


S1=[]
S2=[]
V=set()
G={0: {0}, 1: {1}, 2: {2}, 3: {3}, 4: {4}, 5: {5}, 6: {6}}
black=[False, False, False, False, False, False, False]
[2, 0]


In [206]:
sorted_pi = list(pi)
print(sorted_pi)
for v in sort:
    a = min(pi_inv[v], pi_inv[v + 1])
    b = max(pi_inv[v], pi_inv[v + 1])
    if pi[a] + pi[b] == 1:
        b -= 1
    else:
        a += 1
    sorted_pi[a:b+1] = list(map(lambda x: -x, sorted_pi[a:b+1][::-1]))
    print(a, b, sorted_pi)

[0, -4, -3, -5, 1, 2, -6, 7]
4 6 [0, -4, -3, -5, 6, -2, -1, 7]
6 6 [0, -4, -3, -5, 6, -2, 1, 7]
3 5 [0, -4, -3, 2, -6, 5, 1, 7]
0 3 [-2, 3, 4, 0, -6, 5, 1, 7]
